# RAG System with Unstructured, Qdrant, and OpenAI

This notebook contains code for a Retrieval Augmented Generation (RAG) system. RAG takes in external user-submitted documents to be used as context for Large Language Models (LLM). 

Due to limitations of LLM token sizes and the importance of quality context, a RAG system has additional steps in its pipeline. It must partition the documents into smaller pieces and "chunk" them together. The chunk contains the information needed by the LLM, as well as additional background knowledge, to provide the LLM with the best context. This requires searching which part of the document contains the information, and that is done by turning the chunks into embeddings. All of these steps are features that should be implemented in a RAG system.

## System Overview

- Unstructured - partitions the documents and chunks them together
- embedding models
- Qdrant - vector data store for storing and querying embeddings
- OpenAI - LLM model for producing final response

---

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

UNSTRUCTURED_API_KEY = os.getenv("UNSTRUCTURED_API_KEY")
UNSTRUCTURED_API_URL = os.getenv("UNSTRUCTURED_API_URL")

LOCAL_FILE_INPUT_DIR = os.getenv("LOCAL_FILE_INPUT_DIR")
LOCAL_FILE_OUTPUT_DIR = os.getenv("LOCAL_FILE_OUTPUT_DIR")

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-l6-v2"

## Document Ingestion

This phase uses Unstructued to convert documents into chunks.

Notes:
- Using Unstructured Ingest Python Library because we are batch processing multiple files
- Source connector is to a local directory.
- Destination connector is to the Qdrant database.

In [21]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig
from unstructured_ingest.v2.processes.embedder import EmbedderConfig

pipeline = Pipeline.from_configs(
    context=ProcessorConfig(),
    indexer_config=LocalIndexerConfig(input_path=LOCAL_FILE_INPUT_DIR),
    downloader_config=LocalDownloaderConfig(),
    source_connection_config=LocalConnectionConfig(),
    partitioner_config=PartitionerConfig(
        partition_by_api=True,
        api_key=UNSTRUCTURED_API_KEY,
        partition_endpoint=UNSTRUCTURED_API_URL,
        strategy="hi_res",
        additional_partition_args={
            "split_pdf_page": True,
            "split_pdf_allow_failed": True,
            "split_pdf_concurrency_level": 15
        }
    ),
    # chunker_config=ChunkerConfig(chunking_strategy="by_title"),
    embedder_config=EmbedderConfig(
        embedding_provider="langchain-huggingface",
        embedding_model_name=EMBEDDING_MODEL_NAME,
        embedding_api_key=HUGGINGFACEHUB_API_TOKEN
    ),
    uploader_config=LocalUploaderConfig(output_dir=LOCAL_FILE_OUTPUT_DIR)
)

Overriding of current TracerProvider is not allowed
2024-09-25 08:56:26,753 MainProcess INFO     created index with configs: {"input_path": "data", "recursive": false}, connection configs: {"access_config": "**********"}
2024-09-25 08:56:26,755 MainProcess INFO     Created download with configs: {"download_dir": null}, connection configs: {"access_config": "**********"}
2024-09-25 08:56:26,756 MainProcess INFO     created partition with configs: {"strategy": "hi_res", "ocr_languages": null, "encoding": null, "additional_partition_args": {"split_pdf_page": true, "split_pdf_allow_failed": true, "split_pdf_concurrency_level": 15}, "skip_infer_table_types": null, "fields_include": ["element_id", "text", "type", "metadata", "embeddings"], "flatten_metadata": false, "metadata_exclude": [], "metadata_include": [], "partition_endpoint": "https://api.unstructured.io/general/v0/general", "partition_by_api": true, "api_key": "*******", "hi_res_model_name": null}
2024-09-25 08:56:26,757 MainProces

In [22]:
pipeline.run()

2024-09-25 08:56:32,243 MainProcess INFO     running local pipeline: index (LocalIndexer) -> download (LocalDownloader) -> partition (hi_res) -> embed (langchain-huggingface) -> upload (LocalUploader) with configs: {"reprocess": false, "verbose": false, "tqdm": false, "work_dir": "C:\\Users\\codeu\\.cache\\unstructured\\ingest\\pipeline", "num_processes": 2, "max_connections": null, "raise_on_error": false, "disable_parallelism": false, "preserve_downloads": false, "download_only": false, "re_download": false, "uncompress": false, "iter_delete": false, "delete_cache": false, "otel_endpoint": null, "status": {}}
2024-09-25 08:56:32,393 MainProcess INFO     index finished in 0.0s
2024-09-25 08:56:32,408 MainProcess INFO     calling DownloadStep with 5 docs
2024-09-25 08:56:32,408 MainProcess INFO     processing content async
2024-09-25 08:56:32,409 MainProcess WARNING  async code being run in dedicated thread pool to not conflict with existing event loop: <_WindowsSelectorEventLoop runni

## Creating Qdrant Database

The Qdrant database will be storing the vector embeddings of the document chunks.

In [23]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

COLLECTION_NAME = "resumes"

client = QdrantClient(url="http://localhost:6333")
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=384, distance=Distance.DOT)
)

True

## Storing Vector Embeddings into Qdrant

This function takes a chunk created by Unstructured and inserts it into the Qdrant database.

In [24]:
from qdrant_client.models import PointStruct

def insert_qdrant_point(chunk):
    print(f'Inserting Chunk {chunk["element_id"]}', end="")
    status = client.upsert(
        collection_name=COLLECTION_NAME,
        wait=True,
        points=[
            PointStruct(
                id=chunk["element_id"],
                vector=chunk["embeddings"],
                payload={
                    "text": chunk["text"],
                    "filename": chunk["metadata"]["filename"], 
                }), 
        ]
    )
    print(status)

In [25]:
import json

# Output directory of JSON files
output_dir = "./output"

for filename in os.scandir(output_dir):
    if filename.is_file():
        file = open(filename.path)
        file_json = json.load(file)
        for chunk in file_json:
            insert_qdrant_point(chunk)

Inserting Chunk 6fcee20a4414adcca9d0743d2fa4174coperation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 422bca2e1a097ebaa25daf86cc517df9operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 73a6b890c933664d41b764ba76f27716operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 6b932145122949306692a4c64d0cf656operation_id=3 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 706f65324f496588e14faf79ff0dd946operation_id=4 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk ed2e103de56d612df8fbada5ebe74231operation_id=5 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk b20d0d7cf454f440663f0ac17b47697aoperation_id=6 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 7b871c18de37b63bdba99c6476e09754operation_id=7 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk 1dc452b2ab55cea13fd91951c27ecc2aoperation_id=8 status=<UpdateStatus.COMPLETED: 'completed'>
Inserting Chunk dc2ae95d4999

## Setting up LLM and Embedding Model

The Embedding Model will be used to create the vector embedding for the user query. The resulting embedding will be used to query Qdrant.

The LLM is a HuggingFace ChatModel provided by Langchain. It is used for the text generation.

In [57]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=10000,
    do_sample=False,
)
model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\codeu\.cache\huggingface\token
Login successful


In [27]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN,
    model_name=EMBEDDING_MODEL_NAME,
)

In [50]:
user_query = "Who has experience in Aerospace Engineering? Write a full sentence."

## Querying Qdrant

We can use the embedding model to generate a search vector for Qdrant to get the most similar embedding and retrieve the most relevant document(s).

In [51]:
hits = client.query_points(
    collection_name=COLLECTION_NAME,
    query=embeddings.embed_query(user_query),
).points

retrieved_docs = [hit.payload for hit in hits]
print(len(retrieved_docs))
for doc in retrieved_docs:
    print(doc, end="\n---\n")

10
{'text': 'Profile Mechanical engineer with a knack for propulsion systems and a thirst for knowledge. I bring three years of experience at the intersection of technology and defense, eager to elevate aerospace projects at the Air Force Research Laboratory.', 'filename': 'Brian Patel.txt'}
---
{'text': 'Profile Driven aerospace engineer with a passion for innovation in UAV technology. With over five years of hands-on experience in flight dynamics, I am excited to contribute my expertise to the Air Force Research Laboratory, pushing the boundaries of aerial capabilities.', 'filename': 'Alexandra Johnson.txt'}
---
{'text': 'Education Bachelor’s in Aerospace Engineering University of Washington, Seattle, WA Graduated: June 2023', 'filename': 'Eva Chen.txt'}
---
{'text': 'Profile Recent aerospace engineering graduate with hands-on experience in aerodynamic testing. I’m eager to apply my skills and enthusiasm at the Air Force Research Laboratory, contributing to cutting-edge aerospace pro

## Generating final response from LLM

Now that we have a list of releveant documents, we can append it to the query. The LLM now has more contextual information to give a relevant and correct answer.

In [ ]:
chain = model | parser


In [60]:
instruction = "Here are list of resumes."
for doc in retrieved_docs:
    instruction += "\n" + str(doc)
len(instruction)

2418

In [61]:
response = chain.invoke(instruction + "\n" + user_query)
response

"Brian Patel, Alexandra Johnson, Eva Chen, Chloe Kim, and David Martinez all have experience in Aerospace Engineering, as evidenced by their respective resume entries. \n\n- Brian Patel's resume states that he is a Mechanical engineer with three years of experience at the intersection of technology and defense, specifically related to propulsion systems and defense.\n- Alexandra Johnson's resume indicates that she is an aerospace engineer with over five"

In [59]:
len(response)

453